## Rock style

In [8]:
import qiskit
import numpy as np
from qiskit.quantum_info import Statevector, partial_trace
from music21 import stream, note, instrument, tempo, meter

# Convert a qubit's reduced state to Bloch sphere angles
def state_to_bloch_angles(alpha, beta):
    theta = 2 * np.arccos(np.abs(alpha))
    phi = np.angle(beta) - np.angle(alpha)
    return theta, phi

# Convert Bloch angles to MIDI notes
def theta_to_note(theta):
    pitch = int((theta / np.pi) * 12)
    return pitch + 60  # Middle C (C4)

def phi_to_note(phi):
    pitch = int(((phi + np.pi) / (2 * np.pi)) * 12)
    return pitch + 72  # One octave above middle C

# Assign instruments to qubits (Rock Style)
def qubit_to_instrument(index):
    if index == 0:
        return instrument.ElectricBass()  # Bass guitar for the rhythm section
    if index == 1:
        return instrument.ElectricGuitar()  # Electric guitar for lead
    if index == 2:
        return instrument.SnareDrum()  # Snare drum for rhythmic backbeat
    if index == 3:
        return instrument.BassDrum()  # Bass drum for low end
    if index == 4:
        return instrument.Percussion()  # Percussion for cymbals and other percussive sounds
    if index == 5:
        return instrument.Piano()  # Piano for harmonic support
    instruments = [instrument.Violin(), instrument.Flute(), instrument.Clarinet()]
    return instruments[index % len(instruments)]

# Apply gate to state using circuit reconstruction
def apply_gate(state, gate, qargs, num_qubits):
    temp_circuit = qiskit.QuantumCircuit(num_qubits)
    temp_circuit.append(gate, [temp_circuit.qubits.index(q) for q in qargs])
    return state.evolve(temp_circuit)

# Main function: quantum circuit to music
def circuit_to_music_explicit(circuit):
    num_qubits = circuit.num_qubits
    state = Statevector.from_label('0' * num_qubits)
    qubit_states = {i: [] for i in range(num_qubits)}
    music_score = stream.Score()

    for inst, qargs, _ in circuit.data:
        if inst.name == 'barrier':
            # Record amplitudes at each barrier
            for i in range(num_qubits):
                # Use partial_trace to reduce the state for qubit i
                reduced = partial_trace(state, [j for j in range(num_qubits) if j != i])
                alpha, beta = reduced.data[0][0], reduced.data[1][0]
                qubit_states[i].append((alpha, beta))
        else:
            state = apply_gate(state, inst, qargs, num_qubits)

    # Create the score, inspired by rock music's energy and rhythm
    repeat_count = 8  # Number of repetitions for driving sections

    for i in range(num_qubits):
        part = stream.Part()
        part.insert(0, qubit_to_instrument(i))

        # Bass line (first qubit) as a repetitive, steady rhythm
        if i == 0:
            for _ in range(repeat_count):
                for alpha, beta in qubit_states[i]:
                    theta, phi = state_to_bloch_angles(alpha, beta)
                    note_theta = note.Note(theta_to_note(theta), quarterLength=1)
                    part.append(note_theta)  # Bass plays repetitive, steady notes

            # Now, adding the bass solo
            solo_start = repeat_count  # Start the bass solo after the initial repetitions
            solo_end = solo_start + 8  # Solo section lasts for 8 measures

            for _ in range(solo_start, solo_end):
                for alpha, beta in qubit_states[i]:
                    theta, phi = state_to_bloch_angles(alpha, beta)
                    # Solo uses faster, more varied notes
                    note_theta = note.Note(theta_to_note(theta), quarterLength=0.25)
                    part.append(note_theta)  # Faster, more expressive notes
                    note_phi = note.Note(phi_to_note(phi), quarterLength=0.25)
                    part.append(note_phi)  # Play phi simultaneously for more expressiveness

        # Electric guitar (second qubit) for lead with power chords and some rhythmic variation
        elif i == 1:
            for _ in range(repeat_count):
                for alpha, beta in qubit_states[i]:
                    theta, phi = state_to_bloch_angles(alpha, beta)
                    note_theta = note.Note(theta_to_note(theta), quarterLength=0.5)
                    note_phi = note.Note(phi_to_note(phi), quarterLength=0.5)
                    part.append([note_theta, note_phi])  # Guitar plays power chords with variation
        # Drums (Snare, Bass, Cymbal)
        elif i == 2:
            for _ in range(repeat_count):
                for alpha, beta in qubit_states[i]:
                    theta, phi = state_to_bloch_angles(alpha, beta)
                    note_theta = note.Note(theta_to_note(theta), quarterLength=0.25)
                    part.append(note_theta)  # Percussion for backbeat rhythm
        elif i == 3:
            for _ in range(repeat_count):
                for alpha, beta in qubit_states[i]:
                    theta, phi = state_to_bloch_angles(alpha, beta)
                    note_theta = note.Note(theta_to_note(theta), quarterLength=0.5)
                    part.append(note_theta)  # Bass drum rhythm
        elif i == 4:
            for _ in range(repeat_count):
                for alpha, beta in qubit_states[i]:
                    theta, phi = state_to_bloch_angles(alpha, beta)
                    note_theta = note.Note(theta_to_note(theta), quarterLength=0.25)
                    part.append(note_theta)  # Percussion (Cymbal)
        # Other instruments (e.g., piano, violin) provide harmonic support
        elif i == 5:
            for _ in range(repeat_count):
                for alpha, beta in qubit_states[i]:
                    theta, phi = state_to_bloch_angles(alpha, beta)
                    note_theta = note.Note(theta_to_note(theta), quarterLength=0.75)
                    part.append(note_theta)  # Piano for harmonic background
        music_score.append(part)

    # Set rock tempo and time signature
    music_score.append(meter.TimeSignature('4/4'))  # Common rock time signature
    music_score.append(tempo.MetronomeMark(number=130))  # Fast rock tempo

    return music_score, qubit_states

# Example: build an extended and more varied quantum circuit with 6 qubits
qc = qiskit.QuantumCircuit(6)  # Adding more qubits for complexity
qc.h(0)  # Hadamard gate for qubit 0
qc.cx(0, 1)  # CNOT gate between qubits 0 and 1
qc.barrier()
qc.rz(np.pi/4, 2)  # Rotation for qubit 2
qc.ry(np.pi/2, 1)  # Rotation for qubit 1
qc.rx(np.pi/3, 0)  # Rotation for qubit 0
qc.barrier()
qc.cx(1, 2)  # Add CNOT gate between qubits 1 and 2
qc.rz(np.pi/2, 1)  # Add rotation for qubit 1
qc.barrier()
qc.cx(0, 2)  # Add CNOT gate between qubits 0 and 2
qc.rz(np.pi/3, 2)  # Add rotation for qubit 2
qc.barrier()
qc.ry(np.pi/4, 0)  # Add rotation for qubit 0
qc.cx(0, 1)  # Add CNOT gate between qubits 0 and 1
qc.barrier()
qc.x(1)  # X gate on qubit 1 (flipping it)
qc.barrier()
qc.h(2)  # Hadamard gate for qubit 2
qc.barrier()

# Generate the musical score
music_score, qubit_states = circuit_to_music_explicit(qc)

# Show the score or save it
music_score.show('text')  # Opens MIDI playback in music21
music_score.write('midi', fp='quantum_music_rock_with_bass_solo_extended_with_percussion.mid')  # Save to file if desired


{0.0} <music21.stream.Part 0x1210118e0>
    {0.0} <music21.instrument.ElectricBass 'Electric Bass'>
    {0.0} <music21.note.Note G#>
    {1.0} <music21.note.Note G#>
    {2.0} <music21.note.Note G#>
    {3.0} <music21.note.Note G#>
    {4.0} <music21.note.Note G#>
    {5.0} <music21.note.Note G#>
    {6.0} <music21.note.Note G#>
    {7.0} <music21.note.Note G#>
    {8.0} <music21.note.Note G#>
    {9.0} <music21.note.Note G#>
    {10.0} <music21.note.Note G#>
    {11.0} <music21.note.Note G#>
    {12.0} <music21.note.Note G#>
    {13.0} <music21.note.Note G#>
    {14.0} <music21.note.Note G#>
    {15.0} <music21.note.Note G#>
    {16.0} <music21.note.Note G#>
    {17.0} <music21.note.Note G#>
    {18.0} <music21.note.Note G#>
    {19.0} <music21.note.Note G#>
    {20.0} <music21.note.Note G#>
    {21.0} <music21.note.Note G#>
    {22.0} <music21.note.Note G#>
    {23.0} <music21.note.Note G#>
    {24.0} <music21.note.Note G#>
    {25.0} <music21.note.Note G#>
    {26.0} <music21.note.N

'quantum_music_rock_with_bass_solo_extended_with_percussion.mid'